In [1]:
!pip install -r /kaggle/input/github-benetech-dd/requirements.txt --no-index --find-links=file:///kaggle/input/benetech-sub-pip2 -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.5.0 requires botocore<1.29.77,>=1.29.76, but you have botocore 1.29.157 which is incompatible.


In [2]:
!pip install transformers  --no-index --find-links=file:///kaggle/input/benetech-sub-pip -q 

In [3]:
!cp -r /kaggle/input/github-benetech-dd/* ./

In [4]:
import numpy as np
import pandas as pd
import scipy as sp
import os
import json
import sys
import importlib
import multiprocessing as mp
import gc
from tqdm.auto import tqdm
import glob
import torch
from copy import copy
#from torch.cuda.amp import GradScaler, autocast
from torch.utils.data import DataLoader
import timm
from PIL import Image
import transformers
import torch.nn.functional as F
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
gc.collect()

220

In [5]:
torch.backends.cudnn.benchmark = True

sys.path.append('./configs')
sys.path.append('./data')
sys.path.append('./models')
sys.path.append('./postprocess')
sys.path.append('./scripts')

In [6]:
COMP_FOLDER = '/kaggle/input/benetech-making-graphs-accessible/'
DATA_FOLDER = COMP_FOLDER + 'test/'
META_DF = COMP_FOLDER + 'test.csv'
#train_df = pd.read_csv(COMP_FOLDER + 'train.csv')
#test_df = pd.read_csv(COMP_FOLDER + 'test.csv')
sample_submission = pd.read_csv(COMP_FOLDER + 'sample_submission.csv')
test_files = pd.Series(glob.glob(f'{COMP_FOLDER}/test/images/*.*g')).str.replace('.jpg', '').str.split('/').str[-1]
train_files = pd.Series(glob.glob(f'{COMP_FOLDER}/train/images/*.*g')).str.replace('.jpg', '').str.split('/').str[-1]
test_df = pd.DataFrame({'id':test_files,'chart_type':'scatter','source_type':'extracted','flattened_label':'Jan;Feb;Mar||line||34.1;31.7;31.3||line'})
test_df['fold'] = -1
train_df = pd.DataFrame({'id':train_files,'chart_type':'scatter','source_type':'extracted','flattened_label':'Jan;Feb;Mar||line||34.1;31.7;31.3||line'})
train_df['fold'] = -1
PUBLIC_RUN = len(test_df) == 5
N_CORES = mp.cpu_count()
MIXED_PRECISION = False
PIN_MEMORY = False

RAM_CHECK = False
OOF_CHECK = False
FOLD = 0
MAX_FOLDS = 99
DL_PREFETCH_FACTOR = 2

assert (RAM_CHECK + OOF_CHECK) <= 1

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

if PUBLIC_RUN is False:
    RAM_CHECK = False
    OOF_CHECK = False
    
if RAM_CHECK is True:
    pass
    
if OOF_CHECK is True:
    pass

print(train_df.shape)
print(test_df.shape)
gc.collect()

/tmp/ipykernel_24/153839247.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  test_files = pd.Series(glob.glob(f'{COMP_FOLDER}/test/images/*.*g')).str.replace('.jpg', '').str.split('/').str[-1]
/tmp/ipykernel_24/153839247.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  train_files = pd.Series(glob.glob(f'{COMP_FOLDER}/train/images/*.*g')).str.replace('.jpg', '').str.split('/').str[-1]


(60578, 5)
(5, 5)


32

In [7]:
test_df

,id,chart_type,source_type,flattened_label,fold
0,000b92c3b098,scatter,extracted,Jan;Feb;Mar||line||34.1;31.7;31.3||line,-1
1,01b45b831589,scatter,extracted,Jan;Feb;Mar||line||34.1;31.7;31.3||line,-1
2,00f5404753cf,scatter,extracted,Jan;Feb;Mar||line||34.1;31.7;31.3||line,-1
3,00dcf883a459,scatter,extracted,Jan;Feb;Mar||line||34.1;31.7;31.3||line,-1
4,007a18eb4e09,scatter,extracted,Jan;Feb;Mar||line||34.1;31.7;31.3||line,-1


In [8]:
def get_cfg(CFG):
    cfg = importlib.import_module('default_config')
    importlib.reload(cfg)
    cfg = importlib.import_module(CFG)
    importlib.reload(cfg)
    cfg = copy(cfg.cfg)
    cfg.post_process_pipeline = None #importlib.import_module(cfg.post_process_pipeline).post_process_pipeline

    cfg.data_dir = COMP_FOLDER
    cfg.data_folder = DATA_FOLDER
    cfg.test_data_folder = DATA_FOLDER
    cfg.mixed_precision = MIXED_PRECISION
    cfg.pretrained = False
    cfg.pretrained_weights = False
    cfg.batch_size = cfg.batch_size
    cfg.offline_inference = True

    print(CFG, cfg.model, cfg.dataset, cfg.backbone, cfg.pretrained_weights, cfg.post_process_pipeline)
    
    return cfg

def get_dl(test_df, cfg):
    ds = importlib.import_module(cfg.dataset)
    importlib.reload(ds)

    CustomDataset = ds.CustomDataset
    batch_to_device = ds.batch_to_device

    test_ds = CustomDataset(test_df, cfg, cfg.val_aug, mode="test")
    test_dl = DataLoader(test_ds, shuffle=False, batch_size=cfg.batch_size, collate_fn=ds.val_collate_fn, num_workers=N_CORES, pin_memory=PIN_MEMORY, prefetch_factor = DL_PREFETCH_FACTOR)

    return test_dl, batch_to_device

def get_state_dict(sd_fp):
    sd = torch.load(sd_fp, map_location="cpu")
    if "model" in sd.keys():
        sd = sd["model"]
    sd = {k.replace("module.", ""):v for k,v in sd.items()}
    return sd

def get_nets(cfg,state_dicts):
    model = importlib.import_module(cfg.model)
    importlib.reload(model)
    Net = model.Net

    nets = []

    for i,state_dict in enumerate(state_dicts):
        net = Net(cfg).eval().to(DEVICE)
        print("loading dict")
        sd = get_state_dict(state_dict)
        net.load_state_dict(sd, strict=cfg.pretrained_weights_strict)
        #nets += [net.half()]
        nets += [net]
        del sd
        gc.collect()
    return nets

In [9]:
CFG = 'cfg_ch_ct_1c'
state_dicts = sorted(glob.glob('/kaggle/input/benetech-cfg-ch-ct-1c-ff/*/*.pth'))
print(state_dicts)

cfg = get_cfg(CFG)
# cfg.batch_size = 2
test_dl, batch_to_device = get_dl(test_df, cfg)
nets = get_nets(cfg,state_dicts)

['/kaggle/input/benetech-cfg-ch-ct-1c-ff/fold-1/checkpoint_last_seed309705.pth', '/kaggle/input/benetech-cfg-ch-ct-1c-ff/fold-1/checkpoint_last_seed729826.pth', '/kaggle/input/benetech-cfg-ch-ct-1c-ff/fold-1/checkpoint_last_seed789023.pth']
cfg_ch_ct_1c mdl_ch_4 ds_ch_4 tf_efficientnet_b0_ns False None
scatter    5
Name: chart_type, dtype: int64
loading dict
loading dict
loading dict


In [10]:
with torch.inference_mode():

    preds = []
    for batch in tqdm(test_dl):
        batch = batch_to_device(batch, 'cuda')
        batch_preds = []
        for i, net in enumerate(nets):
            logits = net(batch)['logits'].float().detach().cpu().numpy()
            batch_preds += [logits]
        preds += [np.stack(batch_preds, axis=1)]
    preds = np.concatenate(preds)
    preds.shape

  0%|          | 0/1 [00:00<?, ?it/s]

In [11]:
preds = preds.mean(1).argmax(1)


In [12]:
chart_type_pred = np.array(cfg.chart_map)[preds]

In [13]:
test_df['chart_type_pred'] = chart_type_pred

In [14]:
(test_df['chart_type'] == test_df['chart_type_pred']).mean()

0.2

In [15]:
del nets, preds, test_dl
gc.collect()
torch.cuda.empty_cache()

Exception in thread QueueFeederThread:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/multiprocessing/queues.py", line 239, in _feed
    reader_close()
  File "/opt/conda/lib/python3.10/multiprocessing/connection.py", line 177, in close
    self._close()
  File "/opt/conda/lib/python3.10/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.10/multiprocessing/queues.py", line 271, in _feed
    queue_sem.release()
ValueError: semaphore or lock released too many times


In [16]:
predicted_chart_types = test_df['chart_type_pred'].unique()

In [17]:
predicted_chart_types

array(['line', 'vertical_bar', 'scatter'], dtype=object)

In [18]:
sub_df = []

In [19]:
from typing import Any, Dict

import torch
import torch.nn.functional as F
from torch import nn
from torch.nn.parameter import Parameter
import timm
from torch.distributions import Beta
from torch.nn.utils.rnn import pad_packed_sequence, pad_sequence
from transformers import DetrConfig, DetrForObjectDetection#, DetrHungarianMatcher
from transformers.models.detr.modeling_detr import DetrHungarianMatcher, generalized_box_iou, center_to_corners_format
import torch
import torch
from torch import Tensor
from transformers import DetrImageProcessor
from transformers.models.detr.image_processing_detr import center_to_corners_format
import imagesize
from transformers import Pix2StructForConditionalGeneration, Pix2StructProcessor, AutoTokenizer, Pix2StructConfig
from transformers import Pix2StructVisionModel, T5ForConditionalGeneration
from torch.nn.utils.rnn import pad_sequence

def gem(x, p=3, eps=1e-6):
    return F.avg_pool2d(x.clamp(min=eps).pow(p), (x.size(-2), x.size(-1))).pow(1.0 / p)


class GeM(nn.Module):
    def __init__(self, p=3, eps=1e-6, p_trainable=False):
        super(GeM, self).__init__()
        if p_trainable:
            self.p = Parameter(torch.ones(1) * p)
        else:
            self.p = p
        self.eps = eps

    def forward(self, x):
        ret = gem(x, p=self.p, eps=self.eps)
        return ret

    def __repr__(self):
        return (self.__class__.__name__  + f"(p={self.p.data.tolist()[0]:.4f},eps={self.eps})")


'''
self = Net(cfg)
'''

class Net(nn.Module):

    def __init__(self, cfg: Any):
        super(Net, self).__init__()

        self.cfg = cfg
        
        config = Pix2StructConfig()
        config.vision_config.use_bfloat16 = cfg.use_bfloat16
        config.text_config.use_bfloat16 = cfg.use_bfloat16
        self.backbone = Pix2StructForConditionalGeneration.from_pretrained(cfg.backbone, config = config)
        
        self.processor = Pix2StructProcessor.from_pretrained(cfg.backbone, is_vqa = False)
        sep_pos = self.processor.tokenizer.encode('\;', add_special_tokens = False)
        # wt1 = self.backbone.decoder.embed_tokens.weight[sep_pos]
        wt0 = self.backbone.decoder.embed_tokens.weight#[:-2]
        emb_wt = torch.cat((wt0, wt0[sep_pos]))
        self.backbone.resize_token_embeddings(2+self.backbone.config.text_config.vocab_size)
        self.loss_weights = torch.ones(self.backbone.config.text_config.vocab_size)
        self.loss_weights[-2:] *= self.cfg.break_wt
        self.backbone.decoder.embed_tokens.load_state_dict({'weight': emb_wt})
        
        # self.processor = Pix2StructProcessor.from_pretrained(cfg.backbone)
        self.in_features = self.backbone.config.vision_config.hidden_size
        self.chart_classifier = torch.nn.Linear(in_features=self.in_features, out_features=len(self.cfg.chart_map ), bias=True)
        # self.dtype_classifier = torch.nn.Linear(in_features=self.in_features, out_features=len(self.cfg.dtype_map ), bias=True)
        self.criterion = nn.CrossEntropyLoss(ignore_index=-100)
        calc_grad = False
        for nm, param in self.backbone.named_parameters():
            if cfg.freeze_from in nm:
                calc_grad = True
            param.requires_grad = calc_grad
        # for nm, param in self.backbone.named_parameters():
        #     print(param.requires_grad, nm)
        self.loss_weights = torch.nn.Parameter(self.loss_weights, requires_grad=False)
        # self.cfg.break_wt = 2.
        
        self.criterion_table = nn.CrossEntropyLoss(weight = self.loss_weights , ignore_index=-100, reduction="mean")

    def forward(self, batch):
        
        labels = batch['series_label_1'].permute(1,0)
        encoding = {k:v[:,0] for k,v in batch.items() if k in ['flattened_patches', 'attention_mask']}
        predictions = self.backbone.generate(**encoding, 
                                                max_new_tokens=self.cfg.max_label_length, 
                                                early_stopping=True,
                                                pad_token_id=self.processor.tokenizer.pad_token_id,
                                                eos_token_id=self.processor.tokenizer.eos_token_id,
                                                use_cache=True,
                                                num_beams=self.cfg.num_beams,
                                                temperature=self.cfg.temperature,
                                                top_k=self.cfg.top_k,
                                                return_dict_in_generate=True,
                                                output_hidden_states = True)
        logits_chart = self.chart_classifier(predictions.encoder_hidden_states[-1][:,0])
        logits_table = predictions.sequences #[:,1:]
        # print(logits_table)
        loss = self.criterion(logits_chart, batch['chart_label'])

        outputs = {}
        outputs["loss"] = loss
        if not self.training:
            # Calculate the required padding
            padding = (0, self.cfg.max_label_length - logits_table.size(1))
            # Pad the array along the 3rd dimension
            logits_table  = F.pad(logits_table, padding, mode="constant", value=0)
            outputs["logits_table"] = logits_table
            outputs["logits_chart"] = logits_chart
 
        return outputs


/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [20]:
def get_state_dict(sd_fp):
    sd = torch.load(sd_fp, map_location="cpu")
    if "model" in sd.keys():
        sd = sd["model"]
    sd = {k.replace("module.", ""):v for k,v in sd.items()}
    return sd

def get_nets(cfg,state_dicts):
    model = importlib.import_module(cfg.model)
    importlib.reload(model)
    Net = model.Net

    nets = []

    for i,state_dict in enumerate(state_dicts):
        net = Net(cfg).eval().to(DEVICE)
        print("loading dict")
        sd = get_state_dict(state_dict)
        net.load_state_dict(sd, strict=cfg.pretrained_weights_strict)
        #nets += [net.half()]
        nets += [net]
        del sd
        gc.collect()
    return nets

In [21]:
import torch
import torch.nn.functional as F
from torch import nn
from torch.nn.parameter import Parameter
import torch
from transformers import LogitsProcessorList
from transformers.generation.utils import GreedySearchOutput, GreedySearchEncoderDecoderOutput, GreedySearchDecoderOnlyOutput
from typing import TYPE_CHECKING, Any, Callable, Dict, List, Optional, Tuple, Union
import warnings
from transformers import Pix2StructForConditionalGeneration, Pix2StructProcessor, AutoTokenizer
from transformers import StoppingCriteriaList,  MaxLengthCriteria


        
'''
Reference code:
https://github.com/huggingface/transformers/blob/118e9810687dd713b6be07af79e80eeb1d916908/src/transformers/generation/utils.py#L2164
'''
       


class Pix2StructGreedySearch(Pix2StructForConditionalGeneration):
    def __init__(self, 
                 models, 
                 max_length = 512):
        super().__init__(models[0].config)
        
        del self.encoder 
        del self.decoder 
        self.models = models
        self.is_vqa  = False
        self.logits_processor = LogitsProcessorList([])
        self.stopping_criteria = StoppingCriteriaList([MaxLengthCriteria(max_length=max_length)])
        
        '''
        Reference code:
        https://github.com/huggingface/transformers/blob/118e9810687dd713b6be07af79e80eeb1d916908/src/transformers/generation/utils.py#L2164
        '''
        
    def forward(
        self,
        flattened_patches: Optional[torch.Tensor] = None,
        attention_mask: Optional[torch.Tensor] = None,
        use_cache = True,
        return_dict_in_generate = False,
        output_hidden_states = False,
        
        ):

        if self.models[0].device != flattened_patches.device:
            self.models = [m.to(flattened_patches.device) for m in self.models]
        #print(self.models[0].device)
        #print(flattened_patches.device)

        encoder_outputs_list  = [m.encoder(flattened_patches = flattened_patches, 
                                           attention_mask = attention_mask) for m in self.models]
        input_ids = torch.tensor([[self.models[0].config.pad_token_id]] * len(flattened_patches), 
                                 device = flattened_patches.device) # '<pad>' token to begin
        # attention_mask = attention_mask
        # print(flattened_patches.shape)
        # print(input_ids.shape)
        
        output = self.greedy_search(encoder_outputs_list, 
                                               attention_mask, 
                                               input_ids, 
                                               output_attentions = False,
                                               output_hidden_states = output_hidden_states,
                                               output_scores = False,
                                               return_dict_in_generate =return_dict_in_generate
                                    
                                                
                                   
                                   )
                
        return output

    def greedy_search(
        self,
        encoder_outputs_list,
        attention_mask,
        input_ids: torch.LongTensor,
        stopping_criteria: Optional[StoppingCriteriaList] = None,
        # max_length: Optional[int] = None,
        pad_token_id: Optional[int] = None,
        eos_token_id: Optional[Union[int, List[int]]] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        output_scores: Optional[bool] = None,
        return_dict_in_generate: Optional[bool] = None,
        synced_gpus: bool = False,
        streamer: Optional["BaseStreamer"] = None,
        **model_kwargs,
    ) -> Union[GreedySearchOutput, torch.LongTensor]:
        # init values
        logits_processor = LogitsProcessorList()
        stopping_criteria = self.stopping_criteria
        pad_token_id = self.generation_config.pad_token_id
        eos_token_id = self.generation_config.eos_token_id
        if isinstance(eos_token_id, int):
            eos_token_id = [eos_token_id]
        eos_token_id_tensor = torch.tensor(eos_token_id).to(input_ids.device) if eos_token_id is not None else None
        output_scores = output_scores if output_scores is not None else self.generation_config.output_scores
        output_attentions = (
            output_attentions if output_attentions is not None else self.generation_config.output_attentions
        )
        output_hidden_states = (
            output_hidden_states if output_hidden_states is not None else self.generation_config.output_hidden_states
        )
        return_dict_in_generate = (
            return_dict_in_generate
            if return_dict_in_generate is not None
            else self.generation_config.return_dict_in_generate
        )
    
        # init attention / hidden states / scores tuples
        scores = () if (return_dict_in_generate and output_scores) else None
    
        # keep track of which sequences are already finished
        unfinished_sequences = torch.ones(input_ids.shape[0], dtype=torch.long, device=input_ids.device)
        
        this_peer_finished = False  # used by synced_gpus only
        
        model_kwargs_list = [{'use_cache': True, 
                              'attention_mask': attention_mask, 
                              'encoder_outputs':encoder_outputs} for encoder_outputs in encoder_outputs_list]
        
        decoder_hidden_states_list = [() for _ in range(len(self.models))]
        
        while True:
            # prepare model inputs
            # model_inputs = self.prepare_inputs_for_generation(input_ids, **model_kwargs)
            model_inputs_list = [self.prepare_inputs_for_generation(input_ids, **model_kwargs) for model_kwargs in model_kwargs_list]
            
            outputs_list = []
            for m, model_inputs in zip(self.models, model_inputs_list):
                outputs_list.append(m(
                    **model_inputs,
                    return_dict=True,
                    output_attentions=output_attentions,
                    output_hidden_states=output_hidden_states,
                ))
            
            # next_token_logits = outputs.logits[:, -1, :]
            next_token_logits = torch.stack([o.logits[:, -1, :] for o in outputs_list]).mean(0)
    
            # pre-process distribution
            next_tokens_scores = logits_processor(input_ids, next_token_logits)
            
            # Store scores, attentions and hidden_states when required
            if return_dict_in_generate:
                if output_scores:
                    scores += (next_tokens_scores,)
                
                if output_hidden_states:
                    for ii in range(len(model_kwargs_list)):
                        decoder_hidden_states_list[ii] += (
                            (outputs_list[ii].decoder_hidden_states,)
                            if self.config.is_encoder_decoder
                            else (outputs_list[ii].hidden_states,)
                        )
        
            # argmax
            next_tokens = torch.argmax(next_tokens_scores, dim=-1)
            
            # finished sentences should have their next token be a padding token
            if eos_token_id is not None:
                if pad_token_id is None:
                    raise ValueError("If `eos_token_id` is defined, make sure that `pad_token_id` is defined.")
                next_tokens = next_tokens * unfinished_sequences + pad_token_id * (1 - unfinished_sequences)
            
            # update generated ids, model inputs, and length for next step
            input_ids = torch.cat([input_ids, next_tokens[:, None]], dim=-1)

            for ii in range(len(model_kwargs_list)):
                model_kwargs_list[ii] = self.models[ii]._update_model_kwargs_for_generation(
                                            outputs_list[ii], model_kwargs_list[ii], is_encoder_decoder=self.config.is_encoder_decoder
                                        )
            
            # if eos_token was found in one sentence, set sentence to finished
            if eos_token_id_tensor is not None:
                unfinished_sequences = unfinished_sequences.mul(
                    next_tokens.tile(eos_token_id_tensor.shape[0], 1).ne(eos_token_id_tensor.unsqueeze(1)).prod(dim=0)
                )
    
                # stop when each sentence is finished
                if unfinished_sequences.max() == 0:
                    this_peer_finished = True
    
            # stop if we exceed the maximum length
            if stopping_criteria(input_ids, scores):
                this_peer_finished = True
    
            if this_peer_finished:
                break
            
        return GreedySearchEncoderDecoderOutput(
            sequences=input_ids,
            scores=scores,
            #attentions=decoder_attentions,
            decoder_hidden_states=decoder_hidden_states_list,  # not sure if decoder hidden states if stored correctly, reference original code
            encoder_hidden_states=encoder_outputs_list, 
        )




class EnsembleNet(nn.Module):

    def __init__(self, models, cfg):
        super(EnsembleNet, self).__init__()

        self.cfg = cfg
        
#         for t, wtsnm in enumerate(cfg.pretrained_weights_blend):
#             print(f'Loading {wtsnm}')
#             wts = torch.load(wtsnm, map_location=torch.device('cpu'))
#             if 'model' in wts:
#                 wts = wts['model']
#             models[t].load_state_dict(wts, strict = False)
            
        backbones = [m.backbone for m in models]
        self.chart_classifiers = torch.nn.ModuleList([m.chart_classifier for m in models])
        self.criterion = models[0].criterion
        self.backbones = Pix2StructGreedySearch(backbones, max_length=cfg.max_label_length)
        
    def forward(self, batch):
        
        encoding = {k:v[:,0] for k,v in batch.items() if k in ['flattened_patches', 'attention_mask']}
        
        predictions = self.backbones(**encoding)
        
#         logits_table = predictions.sequences #[:,1:]
        # tokenizer.batch_decode(logits_table)
        
        logits_chart = torch.stack([m(enc.last_hidden_state[:, 0]) for m, enc in zip(self.chart_classifiers, predictions.encoder_hidden_states)]).mean(0)
        
        return predictions, logits_chart


## Scatter 

In [22]:
if 'scatter' in predicted_chart_types:

    test_df_scatter = test_df[test_df['chart_type_pred'] == 'scatter'].copy()
    name = 'cfg_ch_7_2048'

    cfg = get_cfg(name)
    cfg.backbone = '/kaggle/input/benetech-deplot/google/deplot/'
    cfg.data_folder = DATA_FOLDER
    cfg.batch_size = 4
    cfg.model = "mdl_dh_08I_val"
    cfg.post_process_pipeline =  "pp_dh_05B"
    cfg.pretrained_weights_strict = False
    cfg.num_beams=1
    cfg.temperature=1
    cfg.top_k=1
    cfg.max_label_length = 512
#     cfg.post_process_pipeline = 'pp_dh_01J'
    test_dl, batch_to_device = get_dl(test_df_scatter, cfg)
    test_dl.dataset.processor.image_processor.is_vqa = False

    state_dict_fps = sorted(glob.glob('/kaggle/input/benetech-cfg-ch-18-ff/*/check*'))[:2]
    state_dict_fps += sorted(glob.glob('/kaggle/input/benetech-cfg-ch-18b-ff/*/check*'))[:1]
    state_dict_fps += sorted(glob.glob('/kaggle/input/benetech-cfg-ch-18c-ff/*/check*'))[:1]
    
    print('\n'.join(state_dict_fps))
    ens_net = EnsembleNet(get_nets(cfg,state_dict_fps), cfg)

    pp = importlib.import_module(cfg.post_process_pipeline)
    
    print('running inference')
    # https://huggingface.co/docs/transformers/main_classes/text_generation
    logits_charts = []
    logits_tables = []
    with torch.inference_mode():
        for tt, batch in tqdm(enumerate(test_dl), total = len(test_dl)):
            batch = batch_to_device(batch,DEVICE)
            #encoding = {k:v[:,0].half() for k,v in batch.items() if k in ['flattened_patches', 'attention_mask']}
#             encoding = {k:v[:,0] for k,v in batch.items() if k in ['flattened_patches', 'attention_mask']}
#             predictions = nets[0].backbone.generate(**encoding, 
#                                                     max_new_tokens=cfg.max_label_length, 
#                                                     early_stopping=True,
#                                                     pad_token_id=test_dl.dataset.processor.tokenizer.pad_token_id,
#                                                     eos_token_id=test_dl.dataset.processor.tokenizer.eos_token_id,
#                                                     use_cache=True,
#                                                     num_beams=cfg.num_beams,
#                                                     temperature=cfg.temperature,
#                                                     top_k=cfg.top_k,
#                                                     return_dict_in_generate=True,
#                                                     output_hidden_states = True)
            predictions, logits_chart = ens_net(batch)
            logits_table = predictions.sequences
#             logits_chart = nets[0].chart_classifier(predictions.encoder_hidden_states[-1][:,0])
#             logits_table = predictions.sequences
            padding = (0, cfg.max_label_length - predictions.sequences.size(1))
            logits_table  = F.pad(logits_table, padding, mode="constant", value=0)
            logits_charts.append(logits_chart)
            logits_tables.append(logits_table)
    val_data = {}
    val_data['logits_chart'] = torch.cat(logits_charts).cpu()
    val_data['logits_table'] = torch.cat(logits_tables).cpu()
    
    sub_df_scatter = pp.post_process_pipeline(cfg, val_data, test_df_scatter)
#     sub_df += [sub_df_scatter]
    
    del ens_net, val_data, test_dl, logits_chart, logits_table
    gc.collect()
    torch.cuda.empty_cache()

cfg_ch_7_2048 mdl_dh_08O ds_dh_08O google/deplot False None
/kaggle/input/benetech-cfg-ch-18-ff/fold-1/checkpoint_last_seed231088.pth
/kaggle/input/benetech-cfg-ch-18-ff/fold-1/checkpoint_last_seed747128.pth
/kaggle/input/benetech-cfg-ch-18b-ff/fold-1/checkpoint_last_seed298682.pth
/kaggle/input/benetech-cfg-ch-18c-ff/fold-1/checkpoint_last_seed712261.pth
loading dict
loading dict
loading dict
loading dict
running inference


  0%|          | 0/1 [00:00<?, ?it/s]

In [23]:
import albumentations as A

In [24]:
CFG = 'cfg_ch_sunet_1d2_b7_ext_v2_aug0f2_ddp2'
state_dicts = sorted(glob.glob('/kaggle/input/benetech-cfg-ch-sunet-13a-ff/*/*.pth'))
print(state_dicts)

cfg = get_cfg(CFG)
cfg.test_data_folder = cfg.test_data_folder + 'images/'
cfg.val_aug = A.Compose([
    A.Resize(768,768)
#     A.PadIfNeeded (min_height=256, min_width=940),
#     A.LongestMaxSize(cfg.image_width_orig,p=1),
#     A.PadIfNeeded(cfg.image_width_orig, cfg.image_height_orig, border_mode=cv2.BORDER_CONSTANT,p=1),
#     A.CenterCrop(always_apply=False, p=1.0, height=cfg.image_height, width=cfg.image_width), 
#     A.Resize(cfg.img_size[0],cfg.img_size[1])
])
cfg.model = 'mdl_ch_sunet_2_inf'
cfg.return_logits = True
# cfg.batch_size = 2
test_dl, batch_to_device = get_dl(test_df_scatter, cfg)
nets = get_nets(cfg,state_dicts)

['/kaggle/input/benetech-cfg-ch-sunet-13a-ff/fold-1/checkpoint_last_seed123784.pth', '/kaggle/input/benetech-cfg-ch-sunet-13a-ff/fold-1/checkpoint_last_seed571230.pth', '/kaggle/input/benetech-cfg-ch-sunet-13a-ff/fold-1/checkpoint_last_seed665547.pth']
cfg_ch_sunet_1d2_b7_ext_v2_aug0f2_ddp2 mdl_ch_sunet_2 ds_ch_sunet_2_v3 tf_efficientnet_b7_ns False None
loading dict
loading dict
loading dict


In [25]:
with torch.inference_mode():

    n_preds = []
    for batch in tqdm(test_dl):
        batch = batch_to_device(batch, 'cuda')
        batch_preds = []
        for i, net in enumerate(nets):
            x_pred = net(batch)['pred_mask'].float()
            batch_preds += [x_pred]
        batch_preds = torch.stack(batch_preds).mean(0)
        x_pooled = F.max_pool2d(batch_preds, kernel_size=cfg.nms_kernel_size, stride=1, padding=cfg.nms_padding)
        batch_preds[batch_preds != x_pooled] = 0
        n_pred = (batch_preds[:,0] > cfg.n_threshold).sum((1,2)).cpu().numpy()
        n_preds += [n_pred]
    n_preds = np.concatenate(n_preds)

    
del nets, test_dl
gc.collect()
torch.cuda.empty_cache()

  0%|          | 0/1 [00:00<?, ?it/s]

Exception in thread QueueFeederThread:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/multiprocessing/queues.py", line 239, in _feed
    reader_close()
  File "/opt/conda/lib/python3.10/multiprocessing/connection.py", line 177, in close
    self._close()
  File "/opt/conda/lib/python3.10/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.10/multiprocessing/queues.py", line 271, in _feed
    queue_sem.release()
ValueError: semaphore or lock released too many times


In [26]:
def get_mean(x):
    vals = []
    for item in x.split(';'):
        try:
            vals += [float(item)]
        except:
            pass
    if len(vals) > 0:
        m = np.mean(vals)
    else:
        m = 0
    return m

In [27]:
#load unet count preds
oof_df = test_df_scatter[['id']].copy()
oof_df['n_pred'] = n_preds
oof_df_x = oof_df.copy()
oof_df_x['id'] = oof_df_x['id'] + '_x'
oof_df_y = oof_df.copy()
oof_df_y['id'] = oof_df_y['id'] + '_y'
oof_df_xy = pd.concat([oof_df_x,oof_df_y])
oof_df_xy = oof_df_xy.set_index('id')

In [28]:
oof_df_xy.shape, sub_df_scatter.shape

((2, 1), (2, 2))

In [29]:
pp_out_scatter = sub_df_scatter.copy()

#merge into one df
pp_out_scatter['n_current'] = pp_out_scatter['data_series'].apply(lambda x: len(x.split(';')))
pp_out_scatter = pp_out_scatter.merge(oof_df_xy[['n_pred']],how='left',left_index=True,right_index=True)

## count pp1: cut at n_pred if n_current > n_pred

mask = pp_out_scatter['n_current']>pp_out_scatter['n_pred'] #False for nans
if mask.sum()> 0:
    pp_out_scatter.loc[mask,'data_series'] = pp_out_scatter[mask].apply(lambda x: ';'.join((x['data_series'].split(';')[:int(x['n_pred'])])), axis=1)

## count pp2: fill with mean if n_current < n_pred
mask2 = pp_out_scatter['n_current']<pp_out_scatter['n_pred'] #False for nans
if mask2.sum()> 0:
    pp_out_scatter_to_fix = pp_out_scatter.loc[mask2].copy()
    pp_out_scatter_to_fix['mean_data_series'] = pp_out_scatter_to_fix['data_series'].apply(lambda x: get_mean(x))
    pp_out_scatter_to_fix['data_series'] = pp_out_scatter_to_fix.apply(lambda x: ';'.join(x['data_series'].split(';') + [str(x['mean_data_series'])] * (int(x['n_pred']) - x['n_current'])), axis=1)
    pp_out_scatter.loc[mask2, 'data_series'] = pp_out_scatter_to_fix['data_series'].values

sub_df_scatter['data_series'] = pp_out_scatter['data_series'].values

In [30]:
sub_df += [sub_df_scatter]

## Vertical Bar

In [31]:
#do vbar
if 'vertical_bar' in predicted_chart_types:
    
    print('running vertical_bar')
    print('setting up dataloader and model')
    test_df_vb = test_df[test_df['chart_type_pred'] == 'vertical_bar'].copy()
    
    name = 'cfg_ch_7_2048'

    cfg = get_cfg(name)
    cfg.backbone = '/kaggle/input/benetech-deplot/google/deplot/'
    cfg.data_folder = DATA_FOLDER
    cfg.batch_size = 4
    cfg.model = "mdl_dh_08I_val"
    cfg.post_process_pipeline =  "pp_dh_05B_hist"
    cfg.pretrained_weights_strict = False
    cfg.num_beams=1
    cfg.temperature=1
    cfg.top_k=1
    cfg.max_label_length = 512
#     cfg.post_process_pipeline = 'pp_dh_01J'
    test_dl, batch_to_device = get_dl(test_df_vb, cfg)
#     test_dl.dataset.processor.image_processor.is_vqa = False

    state_dict_fps = sorted(glob.glob('/kaggle/input/benetech-cfg-ch-19-ff/*/check*'))[:2]
    state_dict_fps += sorted(glob.glob('/kaggle/input/benetech-cfg-ch-19b-ff/*/check*'))[:2]
    print('\n'.join(state_dict_fps))

    ens_net = EnsembleNet(get_nets(cfg,state_dict_fps), cfg)

    pp = importlib.import_module(cfg.post_process_pipeline)
    
    print('running inference')
    # https://huggingface.co/docs/transformers/main_classes/text_generation
    logits_charts = []
    logits_tables = []
    with torch.inference_mode():
        for tt, batch in tqdm(enumerate(test_dl), total = len(test_dl)):
            batch = batch_to_device(batch,DEVICE)
            #encoding = {k:v[:,0].half() for k,v in batch.items() if k in ['flattened_patches', 'attention_mask']}
#             encoding = {k:v[:,0] for k,v in batch.items() if k in ['flattened_patches', 'attention_mask']}
#             predictions = nets[0].backbone.generate(**encoding, 
#                                                     max_new_tokens=cfg.max_label_length, 
#                                                     early_stopping=True,
#                                                     pad_token_id=test_dl.dataset.processor.tokenizer.pad_token_id,
#                                                     eos_token_id=test_dl.dataset.processor.tokenizer.eos_token_id,
#                                                     use_cache=True,
#                                                     num_beams=cfg.num_beams,
#                                                     temperature=cfg.temperature,
#                                                     top_k=cfg.top_k,
#                                                     return_dict_in_generate=True,
#                                                     output_hidden_states = True)
            predictions, logits_chart = ens_net(batch)
            logits_table = predictions.sequences
#             logits_chart = nets[0].chart_classifier(predictions.encoder_hidden_states[-1][:,0])
#             logits_table = predictions.sequences
            padding = (0, cfg.max_label_length - predictions.sequences.size(1))
            logits_table  = F.pad(logits_table, padding, mode="constant", value=0)
            logits_charts.append(logits_chart)
            logits_tables.append(logits_table)
    val_data = {}
    val_data['logits_chart'] = torch.cat(logits_charts).cpu()
    val_data['logits_table'] = torch.cat(logits_tables).cpu()
    
    
    sub_df_vb = pp.post_process_pipeline(cfg, val_data, test_df_vb)
    sub_df += [sub_df_vb]
    
    
    del ens_net, val_data, test_dl, logits_charts, logits_tables
    gc.collect()
    torch.cuda.empty_cache()

running vertical_bar
setting up dataloader and model
cfg_ch_7_2048 mdl_dh_08O ds_dh_08O google/deplot False None
/kaggle/input/benetech-cfg-ch-19-ff/fold-1/checkpoint_last_seed212431.pth
/kaggle/input/benetech-cfg-ch-19-ff/fold-1/checkpoint_last_seed705191.pth
/kaggle/input/benetech-cfg-ch-19b-ff/fold-1/checkpoint_last_seed20860.pth
/kaggle/input/benetech-cfg-ch-19b-ff/fold-1/checkpoint_last_seed373462.pth
loading dict
loading dict
loading dict
loading dict
running inference


  0%|          | 0/1 [00:00<?, ?it/s]

## Horizontal Bar

In [32]:
if 'horizontal_bar' in predicted_chart_types:
    
    print('running vertical_bar')
    print('setting up dataloader and model')
    test_df_hb = test_df[test_df['chart_type_pred'] == 'horizontal_bar'].copy()
    
    name = 'cfg_ch_7_2048'

    cfg = get_cfg(name)
    cfg.backbone = '/kaggle/input/benetech-deplot/google/deplot/'
    cfg.data_folder = DATA_FOLDER
    cfg.batch_size = 4
    cfg.model = "mdl_dh_08I_val"
    cfg.post_process_pipeline =  "pp_dh_05B"
    cfg.pretrained_weights_strict = False
    cfg.num_beams=1
    cfg.temperature=1
    cfg.top_k=1
    cfg.max_label_length = 512
#     cfg.post_process_pipeline = 'pp_dh_01J'
    test_dl, batch_to_device = get_dl(test_df_hb, cfg)
    test_dl.dataset.processor.image_processor.is_vqa = False

    state_dict_fps = sorted(glob.glob('/kaggle/input/benetech-cfg-ch-18-ff/*/check*'))[:2]
    state_dict_fps += sorted(glob.glob('/kaggle/input/benetech-cfg-ch-18b-ff/*/check*'))[:1]
    state_dict_fps += sorted(glob.glob('/kaggle/input/benetech-cfg-ch-18c-ff/*/check*'))[:1]
    
    print('\n'.join(state_dict_fps))
    ens_net = EnsembleNet(get_nets(cfg,state_dict_fps), cfg)

    pp = importlib.import_module(cfg.post_process_pipeline)
    
    print('running inference')
    # https://huggingface.co/docs/transformers/main_classes/text_generation
    logits_charts = []
    logits_tables = []
    with torch.inference_mode():
        for tt, batch in tqdm(enumerate(test_dl), total = len(test_dl)):
            batch = batch_to_device(batch,DEVICE)
            #encoding = {k:v[:,0].half() for k,v in batch.items() if k in ['flattened_patches', 'attention_mask']}
#             encoding = {k:v[:,0] for k,v in batch.items() if k in ['flattened_patches', 'attention_mask']}
#             predictions = nets[0].backbone.generate(**encoding, 
#                                                     max_new_tokens=cfg.max_label_length, 
#                                                     early_stopping=True,
#                                                     pad_token_id=test_dl.dataset.processor.tokenizer.pad_token_id,
#                                                     eos_token_id=test_dl.dataset.processor.tokenizer.eos_token_id,
#                                                     use_cache=True,
#                                                     num_beams=cfg.num_beams,
#                                                     temperature=cfg.temperature,
#                                                     top_k=cfg.top_k,
#                                                     return_dict_in_generate=True,
#                                                     output_hidden_states = True)
            predictions, logits_chart = ens_net(batch)
            logits_table = predictions.sequences
#             logits_chart = nets[0].chart_classifier(predictions.encoder_hidden_states[-1][:,0])
#             logits_table = predictions.sequences
            padding = (0, cfg.max_label_length - predictions.sequences.size(1))
            logits_table  = F.pad(logits_table, padding, mode="constant", value=0)
            logits_charts.append(logits_chart)
            logits_tables.append(logits_table)
    val_data = {}
    val_data['logits_chart'] = torch.cat(logits_charts).cpu()
    val_data['logits_table'] = torch.cat(logits_tables).cpu()
    
    sub_df_hb = pp.post_process_pipeline(cfg, val_data, test_df_hb)
    sub_df += [sub_df_hb]
    
    del ens_net, val_data, test_dl, logits_charts, logits_tables
    gc.collect()
    torch.cuda.empty_cache()

## Line

In [33]:
if 'line' in predicted_chart_types:

    test_df_line = test_df[test_df['chart_type_pred'] == 'line'].copy()

    
    name = 'cfg_ch_7_2048'

    cfg = get_cfg(name)
    cfg.backbone = '/kaggle/input/benetech-deplot/google/deplot/'
    cfg.data_folder = DATA_FOLDER
    cfg.batch_size = 4
    cfg.model = "mdl_dh_08I_val"
    cfg.post_process_pipeline =  "pp_dh_05B"
    cfg.pretrained_weights_strict = False
    cfg.num_beams=1
    cfg.temperature=1
    cfg.top_k=1
    cfg.max_label_length = 512
#     cfg.post_process_pipeline = 'pp_dh_01J'
    test_dl, batch_to_device = get_dl(test_df_line, cfg)
    test_dl.dataset.processor.image_processor.is_vqa = False

    state_dict_fps = sorted(glob.glob('/kaggle/input/benetech-cfg-ch-18-ff/*/check*'))[:2]
    state_dict_fps += sorted(glob.glob('/kaggle/input/benetech-cfg-ch-18b-ff/*/check*'))[:1]
    state_dict_fps += sorted(glob.glob('/kaggle/input/benetech-cfg-ch-18c-ff/*/check*'))[:1]
    
    print('\n'.join(state_dict_fps))
    ens_net = EnsembleNet(get_nets(cfg,state_dict_fps), cfg)

    pp = importlib.import_module(cfg.post_process_pipeline)
    
    print('running inference')
    # https://huggingface.co/docs/transformers/main_classes/text_generation
    logits_charts = []
    logits_tables = []
    with torch.inference_mode():
        for tt, batch in tqdm(enumerate(test_dl), total = len(test_dl)):
            batch = batch_to_device(batch,DEVICE)
            #encoding = {k:v[:,0].half() for k,v in batch.items() if k in ['flattened_patches', 'attention_mask']}
#             encoding = {k:v[:,0] for k,v in batch.items() if k in ['flattened_patches', 'attention_mask']}
#             predictions = nets[0].backbone.generate(**encoding, 
#                                                     max_new_tokens=cfg.max_label_length, 
#                                                     early_stopping=True,
#                                                     pad_token_id=test_dl.dataset.processor.tokenizer.pad_token_id,
#                                                     eos_token_id=test_dl.dataset.processor.tokenizer.eos_token_id,
#                                                     use_cache=True,
#                                                     num_beams=cfg.num_beams,
#                                                     temperature=cfg.temperature,
#                                                     top_k=cfg.top_k,
#                                                     return_dict_in_generate=True,
#                                                     output_hidden_states = True)
            predictions, logits_chart = ens_net(batch)
            logits_table = predictions.sequences
#             logits_chart = nets[0].chart_classifier(predictions.encoder_hidden_states[-1][:,0])
#             logits_table = predictions.sequences
            padding = (0, cfg.max_label_length - predictions.sequences.size(1))
            logits_table  = F.pad(logits_table, padding, mode="constant", value=0)
            logits_charts.append(logits_chart)
            logits_tables.append(logits_table)
    val_data = {}
    val_data['logits_chart'] = torch.cat(logits_charts).cpu()
    val_data['logits_table'] = torch.cat(logits_tables).cpu()
    
    sub_df_line = pp.post_process_pipeline(cfg, val_data, test_df_line)
    sub_df += [sub_df_line]
    
    del ens_net, val_data, test_dl, logits_charts, logits_tables
    gc.collect()
    torch.cuda.empty_cache()

cfg_ch_7_2048 mdl_dh_08O ds_dh_08O google/deplot False None
/kaggle/input/benetech-cfg-ch-18-ff/fold-1/checkpoint_last_seed231088.pth
/kaggle/input/benetech-cfg-ch-18-ff/fold-1/checkpoint_last_seed747128.pth
/kaggle/input/benetech-cfg-ch-18b-ff/fold-1/checkpoint_last_seed298682.pth
/kaggle/input/benetech-cfg-ch-18c-ff/fold-1/checkpoint_last_seed712261.pth
loading dict
loading dict
loading dict
loading dict
running inference


  0%|          | 0/1 [00:00<?, ?it/s]

## Dot

In [34]:
if 'dot' in predicted_chart_types:
    
    test_df_dot = test_df[test_df['chart_type_pred'] == 'dot'].copy()
    
    name = 'cfg_ch_7_2048'

    cfg = get_cfg(name)
    cfg.backbone = '/kaggle/input/benetech-deplot/google/deplot/'
    cfg.data_folder = DATA_FOLDER
    cfg.batch_size = 4
    cfg.model = "mdl_dh_08I_val"
    cfg.post_process_pipeline =  "pp_dh_05B"
    cfg.pretrained_weights_strict = False
    cfg.num_beams=1
    cfg.temperature=1
    cfg.top_k=1
    cfg.max_label_length = 512
#     cfg.post_process_pipeline = 'pp_dh_01J'
    test_dl, batch_to_device = get_dl(test_df_dot, cfg)
    test_dl.dataset.processor.image_processor.is_vqa = False

    state_dict_fps = sorted(glob.glob('/kaggle/input/benetech-cfg-ch-18-ff/*/check*'))[:2]
    state_dict_fps += sorted(glob.glob('/kaggle/input/benetech-cfg-ch-18b-ff/*/check*'))[:1]
    state_dict_fps += sorted(glob.glob('/kaggle/input/benetech-cfg-ch-18c-ff/*/check*'))[:1]
    
    print('\n'.join(state_dict_fps))
    ens_net = EnsembleNet(get_nets(cfg,state_dict_fps), cfg)

    pp = importlib.import_module(cfg.post_process_pipeline)
    
    print('running inference')
    # https://huggingface.co/docs/transformers/main_classes/text_generation
    logits_charts = []
    logits_tables = []
    with torch.inference_mode():
        for tt, batch in tqdm(enumerate(test_dl), total = len(test_dl)):
            batch = batch_to_device(batch,DEVICE)
            #encoding = {k:v[:,0].half() for k,v in batch.items() if k in ['flattened_patches', 'attention_mask']}
#             encoding = {k:v[:,0] for k,v in batch.items() if k in ['flattened_patches', 'attention_mask']}
#             predictions = nets[0].backbone.generate(**encoding, 
#                                                     max_new_tokens=cfg.max_label_length, 
#                                                     early_stopping=True,
#                                                     pad_token_id=test_dl.dataset.processor.tokenizer.pad_token_id,
#                                                     eos_token_id=test_dl.dataset.processor.tokenizer.eos_token_id,
#                                                     use_cache=True,
#                                                     num_beams=cfg.num_beams,
#                                                     temperature=cfg.temperature,
#                                                     top_k=cfg.top_k,
#                                                     return_dict_in_generate=True,
#                                                     output_hidden_states = True)
            predictions, logits_chart = ens_net(batch)
            logits_table = predictions.sequences
#             logits_chart = nets[0].chart_classifier(predictions.encoder_hidden_states[-1][:,0])
#             logits_table = predictions.sequences
            padding = (0, cfg.max_label_length - predictions.sequences.size(1))
            logits_table  = F.pad(logits_table, padding, mode="constant", value=0)
            logits_charts.append(logits_chart)
            logits_tables.append(logits_table)
    val_data = {}
    val_data['logits_chart'] = torch.cat(logits_charts).cpu()
    val_data['logits_table'] = torch.cat(logits_tables).cpu()
    
    sub_df_dot = pp.post_process_pipeline(cfg, val_data, test_df_dot)
    sub_df += [sub_df_dot]
    
    del ens_net, val_data, test_dl, logits_charts, logits_tables
    gc.collect()
    torch.cuda.empty_cache()

In [35]:
sub_df = pd.concat(sub_df)

In [36]:
sub_df.to_csv("submission.csv")
pd.set_option('display.width', None)
sub_df.head(10)

,data_series,chart_type
id,,
00f5404753cf_x,5;5;5;6;6;6;7;7;7;8;8;8;9;9;9;10;10;10;11;11;1...,scatter
00f5404753cf_y,11;12;14;12;13;14;14;16;17;17;18;19;20;21;22;2...,scatter
00dcf883a459_x,Group 1;Group 2,vertical_bar
00dcf883a459_y,3.6;8.4,vertical_bar
01b45b831589_x,21-Feb;22-Feb;23-Feb;24-Feb;25-Feb;26-Feb;27-F...,vertical_bar
01b45b831589_y,90000;150000;170000;175000;135000;100000;0;400...,vertical_bar
000b92c3b098_x,0;6;12;18;24,line
000b92c3b098_y,0.0;-0.7;-1.4;-2.1;-2.8,line
007a18eb4e09_x,0.0;0.4;0.8;1.2;1.6;2.0;2.4,line


In [37]:
!head submission.csv

id,data_series,chart_type
00f5404753cf_x,5;5;5;6;6;6;7;7;7;8;8;8;9;9;9;10;10;10;11;11;11;11;12;12;12;13;13;13;14;14;14;15;15;15;16;16;16;10.513513513513514,scatter
00f5404753cf_y,11;12;14;12;13;14;14;16;17;17;18;19;20;21;22;21;22;23;21;22;23;24;23;24;26;24;26;27;25;27;28;26;27;29;29;30;31;21.56756756756757,scatter
00dcf883a459_x,Group 1;Group 2,vertical_bar
00dcf883a459_y,3.6;8.4,vertical_bar
01b45b831589_x,21-Feb;22-Feb;23-Feb;24-Feb;25-Feb;26-Feb;27-Feb;28-Feb;29-Feb;01-Mar;02-Mar;03-Mar;04-Mar;05-Mar;06-Mar;07-Mar;08-Mar;09-Mar;10-Mar,vertical_bar
01b45b831589_y,90000;150000;170000;175000;135000;100000;0;40000;60000;65000;55000;45000;65000;80000;80000;105000;130000;100000;10000,vertical_bar
000b92c3b098_x,0;6;12;18;24,line
000b92c3b098_y,0.0;-0.7;-1.4;-2.1;-2.8,line
007a18eb4e09_x,0.0;0.4;0.8;1.2;1.6;2.0;2.4,line


In [38]:
import os

In [39]:
os.listdir('/kaggle/working/')

['configs',
 'postprocess',
 'requirements.txt',
 '__notebook__.ipynb',
 'submission.csv',
 'models',
 'requirements_sub.txt',
 'data']

In [40]:
!rm -rf /kaggle/working/configs/

In [41]:
!rm -rf /kaggle/working/postprocess/

In [42]:
!rm -rf /kaggle/working/data/

In [43]:
!rm -rf /kaggle/working/models/